# Load Postgres database into Katana Graph

This notebook uses a small dataset of 20 nodes with just id values (no other properties exist) and ~350 edges which solely consist of start_id and end_id values (i.e. tuples).

In [ ]:
import os
import psycopg2
import pandas as pd
from katana import remote
from katana.remote import import_data
import dask.dataframe as dd
os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

In [ ]:
graph = remote.Client().create_graph(
    num_partitions=3
)

In [ ]:
conn = psycopg2.connect(
    host="katana-db-1",
    database="test",
    user="postgres",
    password="abc123")

In [ ]:
print(conn)

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM test0")

In [ ]:
test0 = pd.DataFrame(cur.fetchall())

In [ ]:
test0 = test0.rename(columns={0:"test_id"})

In [ ]:
cur.execute("SELECT * FROM test1")

In [ ]:
test1 = pd.DataFrame(cur.fetchall())

In [ ]:
test1 = test1.rename(columns={0:"orig",1:"term"})

In [ ]:
%%time
with import_data.DataFrameImporter(graph) as df_importer:   
    
    df_importer.nodes_dataframe(test0,
                            id_column="test_id",
                            id_space="Test",
                            label="Test")
    
    df_importer.edges_dataframe(test1[['orig', 'term']],
                            source_id_space="Test",
                            destination_id_space="Test",
                            source_column="orig",
                            destination_column="term",
                            type="TEST")

In [ ]:
graph.query("MATCH (n) RETURN n", contextualize=True).view()